# Pretrain

1) Препроцессинг данных

In [ ]:
import re
import json
import hashlib
from pathlib import Path

In [3]:
DATA_PATH = Path("./data/test")
OUT_PATH = Path("./data/pretrain_corpus.jsonl")

MIN_SENT_CHARS = 20
MAX_SENT_CHARS = 5000

CONTEXT_LEN = 1024
MAX_TOKENS_PER_CHUNK = CONTEXT_LEN - 2 

BOS = "<bos>"
EOS = "<eos>"

def normalize_text(text):
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("«", '"').replace("»", '"').replace("„", '"').replace("“", '"').replace("”", '"')
    text = text.replace("—", " — ")
    text = text.replace("–", " — ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def normalize_punct(s):
    s = s.strip()
    s = re.sub(r"\.{4,}", "...", s)
    s = re.sub(r"!{2,}", "!", s)
    s = re.sub(r"\?{2,}", "?", s)
    s = re.sub(r"(\?!){2,}", "?!", s)
    s = re.sub(r",{2,}", ",", s)
    s = re.sub(r":{2,}", ":", s)
    s = re.sub(r";{2,}", ";", s)
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)
    s = re.sub(r"([,.;:!?])([^\s])", r"\1 \2", s)
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

def split_sentences(text):
    parts = re.split(r"(?<=[.!?])\s+", text)
    return [p for p in parts if p.strip()]

LATIN_RE = re.compile(r"[A-Za-z]")
CYR_RE = re.compile(r"[А-Яа-яЁё]")

def cyrillic_ratio(s):
    letters = re.findall(r"[A-Za-zА-Яа-яЁё]", s)
    if not letters:
        return 0.0
    cyr = sum(1 for ch in letters if CYR_RE.match(ch))
    return cyr / len(letters)

def is_good_sentence(s):
    if not s:
        return False

    if len(s) < MIN_SENT_CHARS:
        return False

    if len(s) > MAX_SENT_CHARS:
        return False

    if LATIN_RE.search(s):
        return False
    
    if cyrillic_ratio(s) < 0.70:
        return False
    
    letters_count = len(CYR_RE.findall(s))
    if letters_count < 5:
        return False
    
    if len(set(s)) <= 3:
        return False

    return True

def sha1(text):
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def normalize_for_dedup(s):
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^0-9а-яё]+", "", s)
    return s.strip()

def count_tokens(text, tokenizer=None):
    if tokenizer is None:
        return len(text.split())
    return len(tokenizer.encode(text))

def chunk_sentences(sentences, max_tokens, tokenizer=None):
    chunks = []
    current = []
    current_tokens = 0

    for s in sentences:
        s_tokens = count_tokens(s, tokenizer)
        if s_tokens > max_tokens:
            words = s.split()
            buf = []
            buf_tokens = 0

            for w in words:
                w_tokens = count_tokens(w, tokenizer)
                if buf_tokens + w_tokens > max_tokens and buf:
                    chunks.append(" ".join(buf))
                    buf = [w]
                    buf_tokens = w_tokens
                else:
                    buf.append(w)
                    buf_tokens += w_tokens

            if buf:
                chunks.append(" ".join(buf))
            continue

        if current_tokens + s_tokens > max_tokens and current:
            chunks.append(" ".join(current))
            current = [s]
            current_tokens = s_tokens
        else:
            current.append(s)
            current_tokens += s_tokens
    if current:
        chunks.append(" ".join(current))

    return chunks

In [ ]:
txt_files = sorted(DATA_PATH.glob("*.txt"))
print("Количество файлов:", len(txt_files))

seen_docs = set()
seen_sents = set()
all_chunks = []

stats = {
    "docs_total": 0,
    "docs_unique": 0,
    "sents_total": 0,
    "sents_good": 0,
    "sents_unique": 0,
    "chunks_total": 0
}

for fp in txt_files:
    stats["docs_total"] += 1

    raw = fp.read_text(encoding="utf-8", errors="ignore")
    raw = normalize_text(raw)

    doc_key = sha1(normalize_for_dedup(raw))
    if doc_key in seen_docs:
        continue

    seen_docs.add(doc_key)
    stats["docs_unique"] += 1

    sents = split_sentences(raw)
    stats["sents_total"] += len(sents)

    cleaned = []
    for s in sents:
        s = normalize_punct(s)
        if not is_good_sentence(s):
            continue

        stats["sents_good"] += 1

        sent_key = sha1(normalize_for_dedup(s))
        if sent_key in seen_sents:
            continue

        seen_sents.add(sent_key)
        stats["sents_unique"] += 1
        cleaned.append(s)

    chunks = chunk_sentences(cleaned, max_tokens=MAX_TOKENS_PER_CHUNK, tokenizer=None)

    for ch in chunks:
        text = f"{BOS} {ch.strip()} {EOS}"
        all_chunks.append(text)

stats["chunks_total"] = len(all_chunks)

print("=== STATS ===")
for k, v in stats.items():
    print(f"{k}: {v}")

OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
with OUT_PATH.open("w", encoding="utf-8") as f:
    for t in all_chunks:
        f.write(json.dumps({"text": t}, ensure_ascii=False) + "\n")

print("Saved:", OUT_PATH, "chunks:", len(all_chunks))

Количество файлов: 1
=== STATS ===
docs_total: 1
docs_unique: 1
sents_total: 667
sents_good: 508
sents_unique: 508
chunks_total: 5
Saved: data\pretrain_corpus.jsonl chunks: 5


chunks: 5
min: 769
max: 1024
avg: 970.6
